In [1]:
import pandas as pd
import numpy as np
print("the code is running")
# from Package.Models import *
import torch
import pickle
from torch import nn
from torch.nn.modules.padding import ConstantPad1d,ReflectionPad1d
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.model_selection import train_test_split
from numba import jit
from matplotlib import pyplot as plt
from torch.autograd import Variable
from sklearn.utils import shuffle
from time import time
from textblob import TextBlob as tb
print("import statements finished")

def load_glove_model(gloveFile):
    import numpy as np
    print ("Loading Glove Model")
    f = open(gloveFile,'r',encoding="utf8")
    print("file read")
    model = {}
    count = 0
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        try:
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
        except:
            count = count+1
            pass
    print(count)
    print ("Done.",len(model)," words loaded!")
    
    return model
    
    



# from Package.Features import *
# from Package import Newfile as nf
# global X,Y

## loading the glove vectors
embedding_loc = "../input/embeddings/glove.840B.300d/glove.840B.300d.txt"
# embedding_loc = "../input/quora-insincere-questions-classification/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec"
try:
    glv
except:
    glv = load_glove_model(embedding_loc)
print("trying to load vectors")    
print(len(glv))


the code is running
import statements finished
Loading Glove Model
file read
20
Done. 2195884  words loaded!
trying to load vectors
2195884


In [2]:
DEBUG = True
from nltk.tokenize import TweetTokenizer
import re
tknzr = TweetTokenizer()

torch.manual_seed(2)
def preprocess(t):
    
    t = t.encode("ASCII","ignore").decode("utf-8")
    t = " ".join(tknzr.tokenize(t))
    t = str(t).lower()
    t = re.sub(r'["\'|?"]','',t)
    t = re.sub(r'[\-]',' ',t)
#     print(t)
#     print("^^^^^^^^^^^^^^^^^^")
    return t

def log(*argv):
    if DEBUG == True:
        try:
            print(argv)
        except:
            print("Error in printing")

def get_batches(x,y,chunks = 400):
    l = len(x)
    cnt = int(np.round(int(l)/int(chunks)))
    # log(cnt)
    remain = l%chunks
    rmt = remain/cnt
    # log(rmt)
    x_list = [x[each*cnt:(each*cnt)+cnt] for each in range(chunks+int(rmt)+2)]
    y_list = [y[each*cnt:(each*cnt)+cnt] for each in range(chunks+int(rmt)+2)]
    return x_list,y_list


@jit
def numba_mean(loss_list):
    return np.array(loss_list).mean()
    



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
dtype = torch.FloatTensor
# CUDA_LAUNCH_BLOCKING=1
torch.set_default_tensor_type('torch.FloatTensor')

## Model definition
class encoder_rnn(nn.Module):
    def __init__(self,input_dim, hidden_dim, layer_dim, output_dim):
        super(encoder_rnn,self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True,bidirectional = True)
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim
        self.fc = net = nn.Sequential(
                  nn.Linear(hidden_dim*2, hidden_dim*4),
#                   nn.BatchNorm1d(num_features = (hidden_dim*4)),
                  nn.ReLU(),
                  nn.Linear(hidden_dim*4,output_dim),
                  nn.Sigmoid()
                  )
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(num_features = hidden_dim*4)

    def forward(self,x):
        for e in range(len(x)):
            X = (torch.from_numpy(x[e])[None,None,:]).type(dtype)            
            if e == 0:
#                 h0 = (torch.randn(self.layer_dim*2, 1, self.hidden_dim).type(dtype),torch.randn(self.layer_dim*2, 1, self.hidden_dim).type(dtype))
                h0 = torch.randn(self.layer_dim*2, 1, self.hidden_dim).type(dtype)
                output,hidden = self.rnn(X,h0)
            else:
                output,hidden = self.rnn(X,hidden)
        output = (output).view(-1)
        output = self.relu(output)
        final = self.fc(output)
        return final
def init_weights(m):
    if type(m) == nn.Linear:
        print("Xavier applied")
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)
    
## initiate a model   
model  = encoder_rnn(301,100,2,1)
# torch.nn.init.xavier_uniform(model.weight)
model.apply(init_weights)
# model = model.cuda()

## loading the training data
# q = pd.read_csv("../input/quora-insincere-questions-classification/train.csv",index_col = None,encoding = "iso-8859-1")
q = pd.read_csv("../input/train.csv",index_col = None,encoding = "iso-8859-1")
sin = q[q["target"] == 0]
insin = q[q["target"] == 1]
sin = sin.sample(frac = 0.10,random_state = 42)
q = pd.concat([sin,insin])
q = shuffle(q).reset_index(drop = True)
# q = q[:100000]
log("Total rows ",q.shape)

def get_vectors(text,glv,pad = 250,get_pos = False):
    import nltk
    import numpy as np
    glv_text = " ".join([ each for each in text.split(" ") if each in glv]) 
#     tp = [each for each in glv_text.split(" ")]
#     log(tp)
    all_vectors = [glv[each] for each in glv_text.split(" ")]
    all_vectors = [np.append(all_vectors[i],(tb(each).sentiment[0])) for i,each in enumerate(glv_text.split(" "))]
    temp = np.zeros((301))
    
    if pad != None:
        all_vectors = all_vectors[:pad]
        if len(all_vectors) <pad:
            temp_range = pad-len(all_vectors)
            temp_list = temp.tolist()
            [all_vectors.append(temp_list)for each in range(temp_range)]
    if len(all_vectors)== 0:
        log("The words in this text has no vectors")
        log(text)
        all_vectors.append(temp)
    all_vectors = np.array(all_vectors)
    if get_pos == True:
        pos_vector = get_pos_tag(text,glv)
        if len(all_vectors)!= len(pos_vector):
            print("Pos Error detected")
        all_vectors = [np.concatenate((each,pos_vector[1])) for i,each in enumerate(all_vectors) if len(all_vectors)== len(pos_vector)]
    return all_vectors



log(q.groupby("target")["qid"].count())
log("Considered rows ",q.shape)
q["question_text"] = [ preprocess(each) for each in  q["question_text"]]
# q.preprocess(remove_stopwords = False,lemmatize = False)
log("preprocessing words complete")
q["x"]  = q["question_text"]
q["y"] = q["target"]

## test train split
x,xt,y,yt = train_test_split(q['x'],q['y'],test_size=0.005, random_state=42)
x = x.reset_index(drop = True)
xt = xt.reset_index(drop = True)
y = y.reset_index(drop = True)
yt =yt.reset_index(drop = True)

## readying the vectors for the test data
xt = [get_vectors(each,glv,pad= None) for each in xt]

log("Vectorization complete")

## obtaining batches
batches = 250
x_batches,y_batches = get_batches(x,y,chunks = batches)
x_batches = [each.reset_index(drop = True) for each in x_batches if len(each)!=0]
y_batches = [each.reset_index(drop = True) for each in y_batches if len(each)!=0]

log("splitting the batches is complete")  
log("The shape of each batch is ",x_batches[0].shape)      
log("The shape of test data is ",len(xt))      

def feed(x,y,backprop = False,epoch=1):
    t1 = time()
    loss_list = []
    for i,each in enumerate(x):
        X = each
        Y = (torch.from_numpy(np.array(y[i]))).type(dtype)
        pred = model.forward(X)
        loss =cross(pred,Y).type(dtype)
        if backprop == True:
            loss.backward()
            optimizer.step()
        loss_list.append(loss.item())
    print("T:",str(time()-t1))
    return loss_list

epoch = 1
lr = 0.00001
checkpoint = 3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer.zero_grad()
cross = nn.BCELoss()

log("Model training starts")

for each in range(epoch):
    for j,e in enumerate(x_batches):
        x, y = x_batches[j],y_batches[j]
        x = [get_vectors(each,glv,pad=None) for each in x]
        train_cost = numba_mean(feed(x,y,backprop = True,epoch = each))  
        
        
        log("Epoch num - ",each," and batch num ",j)
        log(train_cost)
        if j%7 == 0:
            test_cost = numba_mean(feed(xt,yt,backprop = False,epoch = each))
            log(test_cost)        
        log("================================================")
        if j == 40:
            break
    break
print("completed")


In [ ]:
# test = pd.read_csv("../input/quora-insincere-questions-classification/test.csv",index_col = None)
test = pd.read_csv("../input/test.csv",index_col = None)
test["question_text"] = [preprocess(each) for each in test["question_text"]]
log("preprocessing words complete")

test["x"]  = test["question_text"]


result = []
for i,each in enumerate(test["x"]):
    # print(each)
    xv = get_vectors(each,glv,pad = None)
    pred =  model.forward(xv)
    result.append(pred.item())
    if i%5000 == 0:
        log(i)
        
#%%
    
test["result"] = result
threshold = 0.4
test["prediction"] = [ 1 if each > threshold else 0 for each in test.result]
test[["qid","prediction"]].to_csv("submission.csv",index = None)
print("submitted")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
